# Listado de Información Demográfica Básica
Información Demográfica Básica: Nombres y Apellidos, Género, Nivel de Educación, Año de Nacimiento.
Información adicional extraída: email.

In [38]:
import pandas as pd

def get_db():
    '''Access MongoDB'''
    from pymongo import MongoClient
    client = MongoClient('mongodb://XXX.XX.X.X:XXXXX')
    db = client.edx
    return db

def get_courses(db):
    '''Get the collections that refers to courses'''
    collections = db.collection_names(include_system_collections=True)
    collections =  list(collections) 
    #print "Collections: \n", collections
    for item in collections: #Remove those that are not courses.
        if item[0:3] != "idb":
            #print item
            collections.remove(item)

    collections.remove("results") #For some reason it does not remove results collection.

    courses = []
    for i in collections:
        c = i.split(".")
        courses.append(c[0])

    courses = set(courses)
    #This is to remove courses that should not be analyzed by the script.
    #Includes test courses and courses never launched.
    #Should only be considered those courses that have been officially launched and closed.
    courses.remove("idb5x_1t2017")
    courses.remove("idb6x_1t2017")
    courses.remove("idb20x_1t2017")
    courses.remove("idb1x_2015_t3")
    courses.remove("idb8_1x_2t2016")
    courses.remove("idb14_1x_1t2017")
    courses.remove("idb14_2x_1t2017")
    courses.remove("idb14x_1t2017")
    courses.remove("idb9x_2015_t3")
    courses.remove("idb3x_1t2017")
    courses.remove("idb6_1x_1t2017")
    courses.remove("idb20_1x_1t2017")
    courses.remove("idb14_3x_1t2017")
    courses.remove("idb2x_2_1t2017")
    courses.remove("idb7x_2t2017")
    courses.remove("idb2x_2t2016")
    courses.remove("idb12x_1t2017")
    courses.remove("idb14x_3t2016")

    
    print "Courses: \n", courses
    return courses, collections

In [39]:
def get_demographics(db, collections):
    '''Get demographic information of the users that have enrolled in each course'''
    #Estadisticas = pd.DataFrame(columns=['Curso','Número de usuarios activos','Total de usuarios registrados','Porcentaje usuarios activos']
                                
    for index, item in enumerate(courses): #Loop for each course
        collection_demographics = item + ".demographics"
        if collection_demographics in collections:
                
            print item
            #Get name of the course from course_names
                                            
            #Get information from demographics
            demographics = db[item].demographics.find({}, {"id":1, "user_id":1, "gender":1, "year_of_birth":1, "level_of_education":1, "country":1}) #Save data in a variable
            df =  pd.DataFrame(list(demographics)) #Transform it into dataframe
            df["course_id"] = item #Add a column with course_id

            user_count = pd.DataFrame(df['user_id'].value_counts()) #Count number of times each user interacts with the platform
            # Prepare variable
            user_count["user_id2"] = user_count.index # Name indexes as a new columnd
            user_count.columns = ["Repetitions", "user_id"] #Name columns

            df2 = pd.merge(df, user_count, on = "user_id", how = "left") #Join original dataframe with counts

            # Get personal information from collection.users
            users = db[item].users.find({"is_staff":0}, {"id":1, "username":1, "first_name":1, "last_name":1, "email":1})
            users_df = pd.DataFrame(list(users))

            df3 = pd.merge(df2, users_df, left_on = "user_id", right_on = "id", how = "left")
            df3 = df3.drop('_id_x', 1)
            df3.rename(columns = {'id_x':'_id'}, inplace = True)
            df3 = df3.drop('_id_y', 1)
            df3 = df3.drop('id_y', 1)
            #print "df3:\n",df3.head(20)
            
            Name_csv = "user_demographics2.csv"
            df3.to_csv(Name_csv, encoding='utf-8')

In [40]:
if __name__=="__main__":
    db = get_db() #Access the db
    courses, collections = get_courses(db) #Access the collections and bring only the courses collections
    get_demographics(db, collections)
    print "HECHO"

Courses: 
set([u'idb9x_2015_t2', u'idb6_3x_3t2016', u'idb14_4x_1t2017', u'idb6x_2016_t1', u'idb6_1x_2016_t2', u'idb5x_2016_t3', u'idb10x_3t2016', u'idb8x_2015_t3', u'idb7x_2t2016', u'idb4_1x_2016_t1', u'idb1x_2016_t2', u'idb1x_2t2014', u'idb6x_2015_t2', u'idb13_1x_3t2016', u'idb2x_2_2t2016', u'idb6_2x_3t2017', u'idb_lsc101x_2015_t4', u'idb10x_2015_t4', u'idb8_1x_2015_t4', u'idb4x_2016_t1', u'idb_lsc101x_3t2014', u'idb3x_2015_t3', u'idb8x_2t2016', u'idb9x_2t2016', u'idb8x_2015_t4', u'idb6x_3t2016', u'idb13x_2t2016', u'idb12x_2016_t2', u'idb1x_2015_3t', u'idb3x_2016_t2', u'idb7x_2015_t1', u'idb2x_3t2014', u'idb4_1x_2015_t4', u'idb5_1x_3t2016'])
idb9x_2015_t2
idb6_3x_3t2016
idb14_4x_1t2017
idb6x_2016_t1
idb6_1x_2016_t2
idb5x_2016_t3
idb10x_3t2016
idb8x_2015_t3
idb7x_2t2016
idb4_1x_2016_t1
idb1x_2016_t2
idb1x_2t2014
idb6x_2015_t2
idb13_1x_3t2016
idb2x_2_2t2016
idb6_2x_3t2017
idb_lsc101x_2015_t4
idb10x_2015_t4
idb8_1x_2015_t4
idb4x_2016_t1
idb_lsc101x_3t2014
idb3x_2015_t3
idb8x_2t2016
idb9x